In [1]:
import pandas as pd

In [7]:
# Load the clean 1min df
df = pd.read_csv('BTC_1min_clean.csv')

# Convert to datetime
df['open_time'] = pd.to_datetime(df['open_time'])

In [8]:
# Function to resample dataframe to a different time frame
def resample_data(df, timeframe='30min'):
    """
    Resamples the DataFrame to a specified time frame.

    Parameters:
    df (DataFrame): The original DataFrame.
    timeframe (str): The resampling time frame.

    Returns:
    DataFrame: The resampled DataFrame.
    """
    df_resampled = df.resample(timeframe, on='open_time').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'quote_vol': 'sum',
        'count': 'sum',
        'buy_base': 'sum',
        'buy_quote': 'sum'
    }).dropna().reset_index()
    
    # Create the new 'close_time' for the resampled data
    df_resampled['close_time'] = df_resampled['open_time'] + pd.to_timedelta(timeframe)
    
    return df_resampled

In [13]:
# Create 30min, 4hr and daily dataframes
df_30min = resample_data(df, timeframe='30min')
df_4hr = resample_data(df, timeframe='240min')
df_daily = resample_data(df, timeframe='1440min')

all_dfs = [df_30min, df_4hr, df_daily]

In [11]:
# Create desire start/end dates
start_date_copy = '2017-09-23' # This date is 1 day early to prevent NaN first row
end_date_copy = '2020-11-30'

In [14]:
# Filter the DataFrame to include data within the specified range
df_30min_trimmed = df_30min[(df_30min['open_time'] >= start_date_copy) & (df_30min['open_time'] <= end_date_copy)]
df_4hr_trimmed = df_4hr[(df_4hr['open_time'] >= start_date_copy) & (df_4hr['open_time'] <= end_date_copy)]
df_daily_trimmed = df_daily[(df_daily['open_time'] >= start_date_copy) & (df_daily['open_time'] <= end_date_copy)]

# Reset the index and drop the old index column
df_30min_trimmed.reset_index(drop=True, inplace=True)
df_4hr_trimmed.reset_index(drop=True, inplace=True)
df_daily_trimmed.reset_index(drop=True, inplace=True)

In [16]:
# Save to CSV
df_30min_trimmed.to_csv("BTC_30min_clean_trimmed.csv", index=False)
df_4hr_trimmed.to_csv("BTC_4h_clean_trimmed.csv", index=False)
df_daily_trimmed.to_csv("BTC_daily_clean_trimmed.csv", index=False)